## Generative Adversarial Networks (GAN)
- A GAN are made of two distinct models, a generator and a discriminator. 
    #### Generator:
    - The job of the generator is to spawn ‘fake’ images that look like the training images

    #### Discriminator:
    - The job of the discriminator is to look at an image and output whether or not it is a real training image or a fake image from the generator

![](assets/GAN_Diagram.png "Generative Adversarial Networks")


### Deep Convultion Genration Adversarial Network (DCGAN):
- A DCGAN is a direct extension of the GAN described above, except that it explicitly uses convolutional and convolutional-transpose layers in the discriminator and generator, respectively

![](assets/DCGAN_Diagram.png "Generative Adversarial Networks")

In [3]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
manualSeed = 49
torch.manual_seed(manualSeed)
